d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Feature Selection Lab

**Objective**: *Apply feature selection to a dataset to derive more meaningful features and improve predictions.*

In this lab, you will apply what you've learned in this lesson. When complete, please use the answers to the exercises to answer questions in the following quiz within Coursera.

In [0]:
%run "../../Includes/Classroom-Setup"

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Out[2]: DataFrame[]

res1: Boolean = false

res2: Boolean = false

res3: Boolean = false

## Exercise 1

In this exercise, you will create a user-level table with the following columns:

1. `avg_resting_heartrate` – the average resting heartrate
1. `avg_active_heartrate` - the average active heartrate
1. `avg_bmi` – the average BMI
1. `avg_vo2` - the average oxygen volume
1. `avg_workout_minutes` - the average of total workout minutes
1. `avg_steps` - the average of total steps
1. `lifestyle` - the lifestyle that best describes the observation

Run the cell below to create the table.

In [0]:
%sql
CREATE OR REPLACE TABLE adsda.ht_user_metrics_pca
USING DELTA LOCATION "/adsda/ht-user-metrics-pca" AS (
  SELECT min(resting_heartrate) AS min_resting_heartrate,
         avg(resting_heartrate) AS avg_resting_heartrate,
         max(resting_heartrate) AS max_resting_heartrate,
         min(active_heartrate) AS min_active_heartrate,
         avg(active_heartrate) AS avg_active_heartrate,
         max(active_heartrate) AS max_active_heartrate,
         avg(bmi) AS avg_bmi,
         min(vo2) AS min_vo2,
         avg(vo2) AS avg_vo2,
         max(vo2) AS max_vo2,
         min(workout_minutes) AS min_workout_minutes,
         avg(workout_minutes) AS avg_workout_minutes,
         max(workout_minutes) AS max_workout_minutes,
         min(steps) AS min_steps,
         avg(steps) AS avg_steps,
         max(steps) AS max_steps,
         avg(steps) * avg(active_heartrate) AS as_x_aah,
         first(lifestyle) AS lifestyle
  FROM adsda.ht_daily_metrics
  GROUP BY device_id
)

num_affected_rows,num_inserted_rows


Run the cell below to convert to a Pandas DataFrame and introduce missing values.

In [0]:
import numpy as np
import pandas as pd
np.random.seed(0)
df = spark.table("adsda.ht_user_metrics_pca").toPandas()
df.loc[df.sample(frac=0.18).index, 'min_active_heartrate'] = np.nan
df.loc[df.sample(frac=0.05).index, 'min_steps'] = np.nan
df.shape

Out[7]: (3000, 18)

## Exercise 2

In this exercise, you'll one-hot encode the `lifestyle` column.

Fill in the blanks below to complete the task.

In [0]:
# TODO
df = pd.get_dummies(df, prefix='dummy', columns=['lifestyle'])

Run this cell to ensure that all columns are numeric.

In [0]:
df = df.apply(pd.to_numeric)

## Exercise 3

In this exercise, you'll split the data into a training set and an inference set.

Fill in the blanks below to complete the task.

In [0]:
# TODO
from sklearn.model_selection import train_test_split

train_df, inference_df = train_test_split(df, train_size=0.85, test_size=0.15, random_state=42)

-sandbox
**Coursera Quiz:** How many rows have missing values in the `min_steps` column in the training set?

Write your code in the empty cell below to answer the question.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** Refer back to the previous lesson for guidance on how to complete this task.

In [0]:
# TODO
train_df.isnull().sum()

Out[13]: min_resting_heartrate        0
avg_resting_heartrate        0
max_resting_heartrate        0
min_active_heartrate       466
avg_active_heartrate         0
max_active_heartrate         0
avg_bmi                      0
min_vo2                      0
avg_vo2                      0
max_vo2                      0
min_workout_minutes          0
avg_workout_minutes          0
max_workout_minutes          0
min_steps                  127
avg_steps                    0
max_steps                    0
as_x_aah                     0
dummy_Athlete                0
dummy_Cardio Enthusiast      0
dummy_Sedentary              0
dummy_Weight Trainer         0
dtype: int64

-sandbox
## Exercise 4

In this exercise, you will fill in these missing values. Using the identified columns from the previous exercise, fill in the missing values with the mean of their respective column.

Fill in the blanks below to complete the task.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** Recall that we want to find the mean of training set and use that to impute values on the training set *and* the test set.

In [0]:
# TODO
mean_active_heartrate = train_df["min_active_heartrate"].mean()
mean_steps = train_df["min_steps"].mean()

train_df["min_active_heartrate"] = mean_active_heartrate
train_df["min_steps"] = mean_steps

inference_df["min_active_heartrate"] = mean_active_heartrate
inference_df["min_steps"] = mean_steps

**Coursera Quiz:** What is the mean of the `min_steps_mean` feature rounded to the nearest hundredth place?

In [0]:
# TODO
round(mean_steps, 2)


Out[16]: 9090.12

## Exercise 5

Create the `X_train`, `X_test`, `y_train`, `y_test` from the train_df. Recall that we are trying to predict the `avg_bmi`.

Fill in the blanks below to complete the task.

In [0]:
# TODO
from sklearn.model_selection import train_test_split

X = train_df.drop("avg_bmi", axis=1)
y = train_df["avg_bmi"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=42)

**Coursera Quiz**: How many rows are in the training set?

In [0]:
X_train.shape

Out[18]: (2295, 20)

## Exercise 6

In this exercise, you will fit a LASSO model. Fill in the blanks to fit a model with a `0.01` alpha, then run the cells to check coefficients.

Fill in the blanks below to complete the task.

In [0]:
# TODO
from sklearn.linear_model import Lasso

lr = Lasso(alpha=.01)
lr.fit(X_train, y_train)

/databricks/python/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:647: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.077e+03, tolerance: 4.789e+00
  model = cd_fast.enet_coordinate_descent(


Out[19]: Lasso(alpha=0.01)

Print out the R^2 score

In [0]:
print(lr.score(X_test, y_test))

0.8993369536891938


**Coursera Quiz**: Which feature had the largest coefficient?

In [0]:
pd.DataFrame(list(zip(lr.coef_, X.columns)), columns=['coef', 'feature_name']).sort_values('coef', ascending=False)

,coef,feature_name
16,4.612639,dummy_Athlete
2,0.078446,max_resting_heartrate
9,0.018358,min_workout_minutes
5,0.012937,max_active_heartrate
14,0.001636,max_steps
15,0.000002,as_x_aah
19,0.000000,dummy_Weight Trainer
3,-0.000000,min_active_heartrate
17,-0.000000,dummy_Cardio Enthusiast
8,-0.000000,max_vo2


## Exercise 7

In this exercise, you will take the feature with the highest coeficients and refit a model.

Fill in the blanks below to complete the task.

In [0]:
# TODO
X = train_df[["dummy_Athlete"]]
y = train_df["avg_bmi"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=42)
lr = Lasso(alpha=.01)
lr.fit(X_train, y_train)

Out[23]: Lasso(alpha=0.01)

Compute the the R-squared score.

Fill in the blanks below to complete the task.

In [0]:
# TODO
lr.score(X_test, y_test)

Out[24]: 0.011697111754633949

Congrats! That concludes our lesson on feature selection!

Be sure to submit your quiz answers to Coursera, and join us in the next lesson to learn about tree based models!

-sandbox
&copy; 2021 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>